In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt
import os
import io
import geopandas as gpd
import seaborn as sns
import statsmodels.api as sm
from functools import reduce
import scipy.stats as stats
from collections import OrderedDict
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import random

# suppress warning

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
# display all columns
# avoid scientific digit
pd.options.display.float_format = '{:.8f}'.format

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
new_df = pd.read_csv('/content/drive/My Drive/X999/new_df.csv')

In [ ]:
new_df.head(1)

,Unnamed: 0,Location,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,eviction_count_nta,eviction_count_building,eviction_count_unit,eviction_count_zipcode,yearbuilt,numfloors,unitsres,bldgarea,bldgclass,building_type,architectural_style,decade,building_category,is_condo,is_llc,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,avg_idr,avg_idr_change,eviction_count_nta.1,2005_median_income,2023_median_income,income_change,income_change_ratio,median_income,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white
0,0,Lower East Side,014679/18_115084,1002600001,014679/18,115084,630 WATER STREET,05E,2019-03-15,MANHATTAN,10002,40.71138100,-73.98286600,3.00000000,2.00000000,202.00000000,1077523,Lower East Side,2019,2019-03,56.72749658,0.60000000,0.00073801,61.00000000,1940.00000000,6.00000000,813.00000000,513000.00000000,D9,0,"1931–1950, Manhattan Modern",1940-1949,elevator,0,0,6.00000000,2.00000000,0.00000000,0.00000000,3.00000000,0.00000000,4.00000000,0.00000000,0.00000000,0.00000000,0.00000000,3.00000000,942.00000000,4.00000000,10.00000000,10.00000000,9.00000000,4.00000000,0.00000000,1.00000000,0.00000000,998.00000000,10.59498500,7.27757147,56.72749658,48525.26513000,54482.00000000,5956.73487000,12.27553287,51503.63256500,76518.00000000,0.96390000,0.87810000,0.93690000,0.92540000,0.96560000,36.80000000,7.60000000,30.00000000,5.40000000,23.10000000,13.10000000,13.00000000,24.70000000,82.80000000,6.80000000,40.80000000,8.00000000,26.00000000,37.20000000,0.10000000,0.00000000,2.40000000,0.80000000,74.40000000,25.60000000


In [ ]:
# new_df['eviction_count_building'] = new_df['eviction_count_building']*5

In [ ]:
new_df.drop(columns=['median_income'], inplace=True)

In [ ]:
new_df.rename(columns={'2023_median_income': 'median_income'}, inplace=True)

In [ ]:
new_df.drop(columns=['2005_median_income'], inplace=True)

In [ ]:
new_df['building_age'] = 2024 - new_df['yearbuilt']

In [ ]:
identifier_columns = ['Location', 'primary_key','bbl','court_index_number','docket_number','eviction_address','eviction_apartment_number',
 'executed_date','borough','zipcode','latitude','longitude','community_board','council_district','census_tract','bin','nta','year','month_year']
metrics_columns = ['eviction_count_nta','eviction_count_building','eviction_count_unit', 'eviction_count_zipcode']
building_columns = ['numfloors', 'unitsres', 'bldgarea', 'bldgclass','building_type', 'architectural_style', "building_age",
                    'building_category', 'is_condo','is_llc']
complaints_columns = ['air_quality', 'animal_issues', 'appliances',  'building_exterior', 'doors_windows', 'electrical_issues', 'elevator_issues',
'floors_stairs','general_complaints', 'graffiti_posting', 'heat_hot_water','homeless_issues', 'noise_complaints','other_issues', 'pest_issues',
'plumbing_issues', 'police_matters', 'public_nuisance','safety_concerns', 'sanitation_issues', 'walls_ceilings', 'total_complaints']
other_features_columns = ['avg_idr','avg_idr_change','income_change','income_change_ratio',
'median_income', 'rpl_theme1', 'rpl_theme2', 'rpl_theme3', 'rpl_theme4', 'rpl_themes', 'ep_pov150', 'ep_unemp', 'ep_nohsdp',
 'ep_uninsur', 'ep_age65', 'ep_age17', 'ep_disabl', 'ep_limeng', 'ep_noveh', 'ep_crowd', 'ep_hburd', 'ep_afam', 'ep_hisp', 'ep_asian',
 'ep_nhpi', 'ep_twomore', 'ep_otherrace', 'ep_minrty', 'ep_white']
# new_df['avg_change_abs']

In [ ]:
# idr_evi.columns.to_list()
all_columns = identifier_columns + metrics_columns + building_columns + complaints_columns + other_features_columns
len(all_columns)

84

In [ ]:
feature_columns = building_columns + complaints_columns + other_features_columns
len(feature_columns)

61

In [ ]:
new_df = new_df[all_columns]
new_df.head(1)

,Location,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,eviction_count_nta,eviction_count_building,eviction_count_unit,eviction_count_zipcode,numfloors,unitsres,bldgarea,bldgclass,building_type,architectural_style,building_age,building_category,is_condo,is_llc,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white
0,Lower East Side,014679/18_115084,1002600001,014679/18,115084,630 WATER STREET,05E,2019-03-15,MANHATTAN,10002,40.71138100,-73.98286600,3.00000000,2.00000000,202.00000000,1077523,Lower East Side,2019,2019-03,56.72749658,0.60000000,0.00073801,61.00000000,6.00000000,813.00000000,513000.00000000,D9,0,"1931–1950, Manhattan Modern",84.00000000,elevator,0,0,6.00000000,2.00000000,0.00000000,0.00000000,3.00000000,0.00000000,4.00000000,0.00000000,0.00000000,0.00000000,0.00000000,3.00000000,942.00000000,4.00000000,10.00000000,10.00000000,9.00000000,4.00000000,0.00000000,1.00000000,0.00000000,998.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,54482.00000000,0.96390000,0.87810000,0.93690000,0.92540000,0.96560000,36.80000000,7.60000000,30.00000000,5.40000000,23.10000000,13.10000000,13.00000000,24.70000000,82.80000000,6.80000000,40.80000000,8.00000000,26.00000000,37.20000000,0.00000000,2.40000000,0.80000000,74.40000000,25.60000000


In [ ]:
new_df.rename(columns={'2023_median_income': 'median_income'}, inplace=True)
other_features_columns = ['avg_idr','avg_idr_change','income_change','income_change_ratio',
'median_income', 'rpl_theme1', 'rpl_theme2', 'rpl_theme3', 'rpl_theme4', 'rpl_themes', 'ep_pov150', 'ep_unemp', 'ep_nohsdp',
 'ep_uninsur', 'ep_age65', 'ep_age17', 'ep_disabl', 'ep_limeng', 'ep_noveh', 'ep_crowd', 'ep_hburd', 'ep_afam', 'ep_hisp', 'ep_asian',
 'ep_nhpi', 'ep_twomore', 'ep_otherrace', 'ep_minrty', 'ep_white']
feature_columns = building_columns + complaints_columns + other_features_columns
len(other_features_columns)

29

In [ ]:
# stop

In [ ]:
new_df[feature_columns].head(1)

,numfloors,unitsres,bldgarea,bldgclass,building_type,architectural_style,building_age,building_category,is_condo,is_llc,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white
0,6.00000000,813.00000000,513000.00000000,D9,0,"1931–1950, Manhattan Modern",84.00000000,elevator,0,0,6.00000000,2.00000000,0.00000000,0.00000000,3.00000000,0.00000000,4.00000000,0.00000000,0.00000000,0.00000000,0.00000000,3.00000000,942.00000000,4.00000000,10.00000000,10.00000000,9.00000000,4.00000000,0.00000000,1.00000000,0.00000000,998.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,54482.00000000,0.96390000,0.87810000,0.93690000,0.92540000,0.96560000,36.80000000,7.60000000,30.00000000,5.40000000,23.10000000,13.10000000,13.00000000,24.70000000,82.80000000,6.80000000,40.80000000,8.00000000,26.00000000,37.20000000,0.00000000,2.40000000,0.80000000,74.40000000,25.60000000


In [ ]:
new_df.bldgclass.unique()

array(['D9', 'D3', 'C1', 'C4', 'C7', 'D1', 'D6', 'C9', 'D7', 'D4', 'S4',
       'C6', 'S5', 'S2', 'S3', 'C5', 'D5', 'R1', 'S9', 'D8', 'R4', 'C2',
       'RR', 'C3', 'C0', 'R2', 'B9', 'A4', 'D2', 'B3', 'B1', 'B2', 'A1',
       'A9', 'S1', 'A5', 'R3', 'A7', 'S0', 'D0', 'R9', 'A2', 'C8', 'A6',
       'A3', 'A8', 'A0', 'R5'], dtype=object)

In [ ]:
new_df.building_category.unique()

array(['elevator', 'walk-up', 'condo-co-op',
       'primarily_res_with_mixed_use', 'two-family', 'single-family'],
      dtype=object)

In [ ]:
unique_strings1 = new_df['bldgclass'].unique()
unique_strings2 = new_df['building_category'].unique()
unique_strings3 = new_df['architectural_style'].unique()
# unique_strings4 = new_df['decade'].unique()

string_to_label1 = {string: random.randint(1, 100) for string in unique_strings1}
string_to_label2 = {string: random.randint(1, 100) for string in unique_strings2}
string_to_label3 = {string: random.randint(1, 100) for string in unique_strings3}
# string_to_label4 = {string: random.randint(1, 100) for string in unique_strings4}

new_df['bldgclass_label'] = new_df['bldgclass'].map(string_to_label1)
new_df['building_category_label'] = new_df['building_category'].map(string_to_label2)
new_df['architectural_style_label'] = new_df['architectural_style'].map(string_to_label3)
# new_df['decade_label'] = new_df['decade'].map(string_to_label)

In [ ]:
# new_df.age.unique()

In [ ]:
# string_to_label4 = {'1940-1949': 1940, '1950-1959':1950, '1960-1969':1960, '1900-1909':1900, '1910-1919':1910, '1980-1989':1980, '1970-1979':1970, '1930-1939':1930, '1920-1929':1920, '1990-1999':1990, '2000-2009':2000, 'Pre-1900':1890, '2010-2020':2010}
# new_df['decade_label'] = new_df['decade'].map(string_to_label4)

In [ ]:
# new_df.building_age.unique()

In [ ]:
new_df.head()

,Location,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,eviction_count_nta,eviction_count_building,eviction_count_unit,eviction_count_zipcode,numfloors,unitsres,bldgarea,bldgclass,building_type,architectural_style,building_age,building_category,is_condo,is_llc,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,bldgclass_label,building_category_label,architectural_style_label
0,Lower East Side,014679/18_115084,1002600001,014679/18,115084,630 WATER STREET,05E,2019-03-15,MANHATTAN,10002,40.71138100,-73.98286600,3.00000000,2.00000000,202.00000000,1077523,Lower East Side,2019,2019-03,56.72749658,0.60000000,0.00073801,61.00000000,6.00000000,813.00000000,513000.00000000,D9,0,"1931–1950, Manhattan Modern",84.00000000,elevator,0,0,6.00000000,2.00000000,0.00000000,0.00000000,3.00000000,0.00000000,4.00000000,0.00000000,0.00000000,0.00000000,0.00000000,3.00000000,942.00000000,4.00000000,10.00000000,10.00000000,9.00000000,4.00000000,0.00000000,1.00000000,0.00000000,998.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,54482.00000000,0.96390000,0.87810000,0.93690000,0.92540000,0.96560000,36.80000000,7.60000000,30.00000000,5.40000000,23.10000000,13.10000000,13.00000000,24.70000000,82.80000000,6.80000000,40.80000000,8.00000000,26.00000000,37.20000000,0.00000000,2.40000000,0.80000000,74.40000000,25.60000000,19,40,54
1,Lower East Side,015506/17_112423,1003560001,015506/17,112423,711 FDR DR,11E,2018-08-22,MANHATTAN,10009,40.72071700,-73.97489500,3.00000000,2.00000000,20.00000000,1078059,Lower East Side,2018,2018-08,56.72749658,0.80000000,0.00042988,67.00000000,14.00000000,1861.00000000,1658649.00000000,D3,1,"1931–1950, Manhattan Modern",74.00000000,elevator,0,0,0.00000000,16.00000000,0.00000000,0.00000000,6.00000000,0.00000000,14.00000000,0.00000000,0.00000000,0.00000000,0.00000000,10.00000000,1021.00000000,1.00000000,24.00000000,13.00000000,4.00000000,11.00000000,1.00000000,58.00000000,0.00000000,1179.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,54482.00000000,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000,50.10000000,85,40,54
2,Lower East Side,0300290/22_5780,1003820001,0300290/22,5780,3 HAVEN PLAZA,15D,2023-05-26,MANHATTAN,10009,40.72682400,-73.97598600,3.00000000,2.00000000,28.00000000,1083144,Lower East Side,2023,2023-05,56.72749658,0.80000000,0.00215633,67.00000000,26.00000000,371.00000000,453600.00000000,D9,1,"1951–1980, the International Style, Alternative Modernism",58.00000000,elevator,0,0,2.00000000,1.00000000,4.00000000,0.00000000,15.00000000,15.00000000,26.00000000,6.00000000,11.00000000,0.00000000,92.00000000,0.00000000,90.00000000,0.00000000,4.00000000,31.00000000,0.00000000,1.00000000,2.00000000,32.00000000,12.00000000,344.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,54482.00000000,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000

In [ ]:
new_df.drop(columns=['bldgclass', 'building_category', 'architectural_style'], inplace=True)

In [ ]:
new_df.rename(columns={'bldgclass_label': 'bldgclass', 'building_category_label': 'building_category', \
                       'architectural_style_label': 'architectural_style'}, inplace=True)

In [ ]:
new_df.head()

,Location,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,eviction_count_nta,eviction_count_building,eviction_count_unit,eviction_count_zipcode,numfloors,unitsres,bldgarea,building_type,building_age,is_condo,is_llc,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,bldgclass,building_category,architectural_style
0,Lower East Side,014679/18_115084,1002600001,014679/18,115084,630 WATER STREET,05E,2019-03-15,MANHATTAN,10002,40.71138100,-73.98286600,3.00000000,2.00000000,202.00000000,1077523,Lower East Side,2019,2019-03,56.72749658,0.60000000,0.00073801,61.00000000,6.00000000,813.00000000,513000.00000000,0,84.00000000,0,0,6.00000000,2.00000000,0.00000000,0.00000000,3.00000000,0.00000000,4.00000000,0.00000000,0.00000000,0.00000000,0.00000000,3.00000000,942.00000000,4.00000000,10.00000000,10.00000000,9.00000000,4.00000000,0.00000000,1.00000000,0.00000000,998.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,54482.00000000,0.96390000,0.87810000,0.93690000,0.92540000,0.96560000,36.80000000,7.60000000,30.00000000,5.40000000,23.10000000,13.10000000,13.00000000,24.70000000,82.80000000,6.80000000,40.80000000,8.00000000,26.00000000,37.20000000,0.00000000,2.40000000,0.80000000,74.40000000,25.60000000,19,40,54
1,Lower East Side,015506/17_112423,1003560001,015506/17,112423,711 FDR DR,11E,2018-08-22,MANHATTAN,10009,40.72071700,-73.97489500,3.00000000,2.00000000,20.00000000,1078059,Lower East Side,2018,2018-08,56.72749658,0.80000000,0.00042988,67.00000000,14.00000000,1861.00000000,1658649.00000000,1,74.00000000,0,0,0.00000000,16.00000000,0.00000000,0.00000000,6.00000000,0.00000000,14.00000000,0.00000000,0.00000000,0.00000000,0.00000000,10.00000000,1021.00000000,1.00000000,24.00000000,13.00000000,4.00000000,11.00000000,1.00000000,58.00000000,0.00000000,1179.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,54482.00000000,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000,50.10000000,85,40,54
2,Lower East Side,0300290/22_5780,1003820001,0300290/22,5780,3 HAVEN PLAZA,15D,2023-05-26,MANHATTAN,10009,40.72682400,-73.97598600,3.00000000,2.00000000,28.00000000,1083144,Lower East Side,2023,2023-05,56.72749658,0.80000000,0.00215633,67.00000000,26.00000000,371.00000000,453600.00000000,1,58.00000000,0,0,2.00000000,1.00000000,4.00000000,0.00000000,15.00000000,15.00000000,26.00000000,6.00000000,11.00000000,0.00000000,92.00000000,0.00000000,90.00000000,0.00000000,4.00000000,31.00000000,0.00000000,1.00000000,2.00000000,32.00000000,12.00000000,344.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,54482.00000000,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000,50.10000000,19,40,56
3,Lower East Side,0301867/23_6498,1003820001,0301867/23,6498,1 HAVEN PLAZA,27C,2023-06-20,MANHATTAN,10009,40.72701600,-73.97644000,3.00000000,2.00000000,28.00000000,1083141,Lower East Side,2023,2023-0

In [ ]:
new_df.columns

Index(['Location', 'primary_key', 'bbl', 'court_index_number', 'docket_number',
       'eviction_address', 'eviction_apartment_number', 'executed_date',
       'borough', 'zipcode', 'latitude', 'longitude', 'community_board',
       'council_district', 'census_tract', 'bin', 'nta', 'year', 'month_year',
       'eviction_count_nta', 'eviction_count_building', 'eviction_count_unit',
       'eviction_count_zipcode', 'numfloors', 'unitsres', 'bldgarea',
       'building_type', 'building_age', 'is_condo', 'is_llc', 'air_quality',
       'animal_issues', 'appliances', 'building_exterior', 'doors_windows',
       'electrical_issues', 'elevator_issues', 'floors_stairs',
       'general_complaints', 'graffiti_posting', 'heat_hot_water',
       'homeless_issues', 'noise_complaints', 'other_issues', 'pest_issues',
       'plumbing_issues', 'police_matters', 'public_nuisance',
       'safety_concerns', 'sanitation_issues', 'walls_ceilings',
       'total_complaints', 'avg_idr', 'avg_idr_change', '

## **Groupby nta on complaint features**

In [ ]:
complaints_columns

['air_quality',
 'animal_issues',
 'appliances',
 'building_exterior',
 'doors_windows',
 'electrical_issues',
 'elevator_issues',
 'floors_stairs',
 'general_complaints',
 'graffiti_posting',
 'heat_hot_water',
 'homeless_issues',
 'noise_complaints',
 'other_issues',
 'pest_issues',
 'plumbing_issues',
 'police_matters',
 'public_nuisance',
 'safety_concerns',
 'sanitation_issues',
 'walls_ceilings',
 'total_complaints']

In [ ]:
BIN_df = new_df.groupby('bin')[complaints_columns].sum().reset_index()

In [ ]:
BIN_df.head()

,bin,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints
0,1000793,0.00000000,18.00000000,0.00000000,18.00000000,27.00000000,180.00000000,378.00000000,36.00000000,54.00000000,0.00000000,864.00000000,27.00000000,1404.00000000,0.00000000,9.00000000,423.00000000,36.00000000,18.00000000,27.00000000,198.00000000,45.00000000,3762.00000000
1,1000810,0.00000000,0.00000000,2.00000000,0.00000000,12.00000000,4.00000000,6.00000000,6.00000000,12.00000000,0.00000000,76.00000000,0.00000000,110.00000000,0.00000000,0.00000000,8.00000000,2.00000000,4.00000000,2.00000000,8.00000000,10.00000000,262.00000000
2,1000816,2.00000000,1.00000000,1.00000000,0.00000000,2.00000000,1.00000000,3.00000000,0.00000000,5.00000000,0.00000000,0.00000000,0.00000000,110.00000000,0.00000000,1.00000000,17.00000000,1.00000000,0.00000000,6.00000000,1.00000000,0.00000000,151.00000000
3,1000826,0.00000000,2.00000000,0.00000000,0.00000000,0.00000000,0.00000000,1.00000000,0.00000000,0.00000000,0.00000000,0.00000000,4.00000000,77.00000000,0.00000000,0.00000000,3.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,87.00000000
4,1000828,8.00000000,8.00000000,8.00000000,0.00000000,24.00000000,0.00000000,128.00000000,0.00000000,32.00000000,0.00000000,96.00000000,0.00000000,1368.00000000,0.00000000,16.00000000,96.00000000,0.00000000,0.00000000,8.00000000,16.00000000,0.00000000,1808.00000000


In [ ]:
BIN_df = new_df.merge(BIN_df, on='bin', how='left', suffixes=('', '_BIN'))

In [ ]:
BIN_df.head()

,Location,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,eviction_count_nta,eviction_count_building,eviction_count_unit,eviction_count_zipcode,numfloors,unitsres,bldgarea,building_type,building_age,is_condo,is_llc,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,bldgclass,building_category,architectural_style,air_quality_BIN,animal_issues_BIN,appliances_BIN,building_exterior_BIN,doors_windows_BIN,electrical_issues_BIN,elevator_issues_BIN,floors_stairs_BIN,general_complaints_BIN,graffiti_posting_BIN,heat_hot_water_BIN,homeless_issues_BIN,noise_complaints_BIN,other_issues_BIN,pest_issues_BIN,plumbing_issues_BIN,police_matters_BIN,public_nuisance_BIN,safety_concerns_BIN,sanitation_issues_BIN,walls_ceilings_BIN,total_complaints_BIN
0,Lower East Side,014679/18_115084,1002600001,014679/18,115084,630 WATER STREET,05E,2019-03-15,MANHATTAN,10002,40.71138100,-73.98286600,3.00000000,2.00000000,202.00000000,1077523,Lower East Side,2019,2019-03,56.72749658,0.60000000,0.00073801,61.00000000,6.00000000,813.00000000,513000.00000000,0,84.00000000,0,0,6.00000000,2.00000000,0.00000000,0.00000000,3.00000000,0.00000000,4.00000000,0.00000000,0.00000000,0.00000000,0.00000000,3.00000000,942.00000000,4.00000000,10.00000000,10.00000000,9.00000000,4.00000000,0.00000000,1.00000000,0.00000000,998.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,54482.00000000,0.96390000,0.87810000,0.93690000,0.92540000,0.96560000,36.80000000,7.60000000,30.00000000,5.40000000,23.10000000,13.10000000,13.00000000,24.70000000,82.80000000,6.80000000,40.80000000,8.00000000,26.00000000,37.20000000,0.00000000,2.40000000,0.80000000,74.40000000,25.60000000,19,40,54,18.00000000,6.00000000,0.00000000,0.00000000,9.00000000,0.00000000,12.00000000,0.00000000,0.00000000,0.00000000,0.00000000,9.00000000,2826.00000000,12.00000000,30.00000000,30.00000000,27.00000000,12.00000000,0.00000000,3.00000000,0.00000000,2994.00000000
1,Lower East Side,015506/17_112423,1003560001,015506/17,112423,711 FDR DR,11E,2018-08-22,MANHATTAN,10009,40.72071700,-73.97489500,3.00000000,2.00000000,20.00000000,1078059,Lower East Side,2018,2018-08,56.72749658,0.80000000,0.00042988,67.00000000,14.00000000,1861.00000000,1658649.00000000,1,74.00000000,0,0,0.00000000,16.00000000,0.00000000,0.00000000,6.00000000,0.00000000,14.00000000,0.00000000,0.00000000,0.00000000,0.00000000,10.00000000,1021.00000000,1.00000000,24.00000000,13.00000000,4.00000000,11.00000000,1.00000000,58.00000000,0.00000000,1179.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,54482.00000000,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000,50.10000000,85,40,54,0.00000000,64.00000000,0.00000000,0.00000000,24.00000000,0.00000000,56.00000000,0.00000000,0.00000000,0.00000000,0.00000000,40.00000000,4084.00000000,4.00000000,96.00000000,52.00000000,16.00000000,44.00000000,4.00000000,232.00000000,0.00000000,4716.00000000
2,Lower East Side,0300290/22_5780,1003820001,0300290/22,5780,3 HAVEN PLAZA,15D,2023-05-26,MANHATTAN,10009,40.72682400,-73.97598600,3.00000000,2.00000000,28.00000000,108

In [ ]:
BIN_df = BIN_df.drop(columns= complaints_columns)
# remeber to drop

In [ ]:
BIN_df.rename(columns={'air_quality_BIN': 'air_quality', 'animal_issues_BIN': 'animal_issues', 'appliances_BIN': 'appliances', \
                           'building_exterior_BIN': 'building_exterior', 'doors_windows_BIN': 'doors_windows', 'electrical_issues_BIN': 'electrical_issues', \
                           'elevator_issues_BIN': 'elevator_issues', 'floors_stairs_BIN': 'floors_stairs', 'general_complaints_BIN': 'general_complaints', \
                           'graffiti_posting_BIN': 'graffiti_posting', 'heat_hot_water_BIN': 'heat_hot_water', 'homeless_issues_BIN': 'homeless_issues', \
                           'noise_complaints_BIN': 'noise_complaints', 'other_issues_BIN': 'other_issues', 'pest_issues_BIN': 'pest_issues', \
                           'plumbing_issues_BIN': 'plumbing_issues', 'police_matters_BIN': 'police_matters', 'public_nuisance_BIN': 'public_nuisance', \
                           'safety_concerns_BIN': 'safety_concerns', 'sanitation_issues_BIN': 'sanitation_issues', 'walls_ceilings_BIN': 'walls_ceilings', \
                           'total_complaints_BIN': 'total_complaints'}, inplace=True)

In [ ]:
# BIN_df[complaints_columns] = BIN_df[complaints_columns].div(5, axis=0)

In [ ]:
BIN_df.head()

,Location,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,eviction_count_nta,eviction_count_building,eviction_count_unit,eviction_count_zipcode,numfloors,unitsres,bldgarea,building_type,building_age,is_condo,is_llc,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,bldgclass,building_category,architectural_style,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints
0,Lower East Side,014679/18_115084,1002600001,014679/18,115084,630 WATER STREET,05E,2019-03-15,MANHATTAN,10002,40.71138100,-73.98286600,3.00000000,2.00000000,202.00000000,1077523,Lower East Side,2019,2019-03,56.72749658,0.60000000,0.00073801,61.00000000,6.00000000,813.00000000,513000.00000000,0,84.00000000,0,0,10.59498500,7.27757147,5956.73487000,12.27553287,54482.00000000,0.96390000,0.87810000,0.93690000,0.92540000,0.96560000,36.80000000,7.60000000,30.00000000,5.40000000,23.10000000,13.10000000,13.00000000,24.70000000,82.80000000,6.80000000,40.80000000,8.00000000,26.00000000,37.20000000,0.00000000,2.40000000,0.80000000,74.40000000,25.60000000,19,40,54,18.00000000,6.00000000,0.00000000,0.00000000,9.00000000,0.00000000,12.00000000,0.00000000,0.00000000,0.00000000,0.00000000,9.00000000,2826.00000000,12.00000000,30.00000000,30.00000000,27.00000000,12.00000000,0.00000000,3.00000000,0.00000000,2994.00000000
1,Lower East Side,015506/17_112423,1003560001,015506/17,112423,711 FDR DR,11E,2018-08-22,MANHATTAN,10009,40.72071700,-73.97489500,3.00000000,2.00000000,20.00000000,1078059,Lower East Side,2018,2018-08,56.72749658,0.80000000,0.00042988,67.00000000,14.00000000,1861.00000000,1658649.00000000,1,74.00000000,0,0,10.59498500,7.27757147,5956.73487000,12.27553287,54482.00000000,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000,50.10000000,85,40,54,0.00000000,64.00000000,0.00000000,0.00000000,24.00000000,0.00000000,56.00000000,0.00000000,0.00000000,0.00000000,0.00000000,40.00000000,4084.00000000,4.00000000,96.00000000,52.00000000,16.00000000,44.00000000,4.00000000,232.00000000,0.00000000,4716.00000000
2,Lower East Side,0300290/22_5780,1003820001,0300290/22,5780,3 HAVEN PLAZA,15D,2023-05-26,MANHATTAN,10009,40.72682400,-73.97598600,3.00000000,2.00000000,28.00000000,1083144,Lower East Side,2023,2023-05,56.72749658,0.80000000,0.00215633,67.00000000,26.00000000,371.00000000,453600.00000000,1,58.00000000,0,0,10.59498500,7.27757147,5956.73487000,12.27553287,54482.00000000,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000,50.10000000,19,40,56,8.00000000,4.00000000,16.00000000,0.00000000,60.00000000,60.00000000,104.00000000,24.00000000,44.00000000,0.00000000,368.00000000,0.00000000,360.00000000,0.00000000,16.00000000,124.00000000,0.00000000,4.00000000,8.00000000,128.00000000,48.00000000,1376.00000000
3,Lower East Side,0301867/23_6498,1003820001,0301867/23,6498,1 HAVEN PLAZA,27C,2023-06-20,MANHATTAN,10009,40.72701600,-73.97644000,3.00000000,2.00000000,28.00000000,1083141,Lower Ea

In [ ]:
BIN_df[complaints_columns] = BIN_df[complaints_columns].div(5, axis=0)

In [ ]:
BIN_df.shape

(66395, 84)

In [ ]:
# stop

In [ ]:
BIN_df.to_csv('/content/drive/My Drive/X999/BIN_df.csv', index=False)

## **BIN_df normalized**

In [ ]:
scaler = StandardScaler()

In [ ]:
# pd.DataFrame(BIN_df_normalized).columns.to_list()
BIN_df.shape

(66395, 84)

In [ ]:
# noramlized BIN_df's feature columns
columns = building_columns + complaints_columns + other_features_columns
print(len(columns))
BIN_df_normalized = scaler.fit_transform(BIN_df[columns])
print(BIN_df[columns].shape)
# BIN_df_normalized = pd.DataFrame(BIN_df_normalized, columns=columns)
BIN_df_normalized = pd.DataFrame(BIN_df_normalized[:, :63], columns=columns)

61
(66395, 61)


In [ ]:
print(BIN_df_normalized.shape)
print(len(columns))

(66395, 61)
61


In [ ]:
BIN_df_normalized = pd.DataFrame(BIN_df_normalized, columns=columns)

In [ ]:
BIN_df_normalized.head(1)

,numfloors,unitsres,bldgarea,bldgclass,building_type,architectural_style,building_age,building_category,is_condo,is_llc,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white
0,-0.06971794,0.81877905,0.36540890,-1.28156686,-0.71188874,0.99501973,0.00370864,0.51460360,-0.25457296,-0.86331212,0.21727361,-0.09594390,-0.09638869,-0.18987319,-0.17210392,-0.19165351,-0.09752602,-0.20015991,-0.14005238,-0.11639498,-0.20184605,0.22882237,0.47887445,4.19344796,-0.04273580,-0.12502628,0.03645647,0.07376326,-0.18270494,-0.14644529,-0.15436581,-0.04282892,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.38793712,0.33656949,0.39990329,-0.13447419,-0.08025525,0.31459399,0.37685617,-0.56794790,1.07678865,-0.68257460,2.20956395,-1.80374947,0.02659191,1.40547024,1.27492190,-0.79928722,-0.18133595,-0.95824164,-0.55596050,2.41678554,-0.25158118,-0.28234398,-0.15525459,-0.32955230,0.32955230


In [ ]:
df = BIN_df['bin']
df.head()

,bin
0,1077523
1,1078059
2,1083144
3,1083141
4,1005824


In [ ]:
identifier_columns = ['Location', 'primary_key','bbl','court_index_number','docket_number','eviction_address','eviction_apartment_number',
 'executed_date','borough','zipcode','latitude','longitude','community_board','council_district','census_tract','bin','nta','year','month_year']
metrics_columns = ['eviction_count_BIN','eviction_count_building','eviction_count_unit', 'eviction_count_zipcode']

In [ ]:
BIN_df_normalized['bin'] = BIN_df['bin']
BIN_df_normalized[['Location', 'primary_key','bbl','court_index_number','docket_number','eviction_address','eviction_apartment_number','executed_date','borough','zipcode','latitude','longitude','community_board','council_district','census_tract','bin','nta','year',
                      'month_year','eviction_count_nta','eviction_count_building','eviction_count_unit', 'eviction_count_zipcode']] = BIN_df[['Location', 'primary_key','bbl','court_index_number','docket_number','eviction_address','eviction_apartment_number','executed_date','borough','zipcode','latitude','longitude','community_board','council_district','census_tract','bin','nta','year',
                      'month_year','eviction_count_nta','eviction_count_building','eviction_count_unit', 'eviction_count_zipcode']]
BIN_df_normalized.head()

,numfloors,unitsres,bldgarea,bldgclass,building_type,architectural_style,building_age,building_category,is_condo,is_llc,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,bin,Location,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,latitude,longitude,community_board,council_district,census_tract,nta,year,month_year,eviction_count_nta,eviction_count_building,eviction_count_unit,eviction_count_zipcode
0,-0.06971794,0.81877905,0.36540890,-1.28156686,-0.71188874,0.99501973,0.00370864,0.51460360,-0.25457296,-0.86331212,0.21727361,-0.09594390,-0.09638869,-0.18987319,-0.17210392,-0.19165351,-0.09752602,-0.20015991,-0.14005238,-0.11639498,-0.20184605,0.22882237,0.47887445,4.19344796,-0.04273580,-0.12502628,0.03645647,0.07376326,-0.18270494,-0.14644529,-0.15436581,-0.04282892,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.38793712,0.33656949,0.39990329,-0.13447419,-0.08025525,0.31459399,0.37685617,-0.56794790,1.07678865,-0.68257460,2.20956395,-1.80374947,0.02659191,1.40547024,1.27492190,-0.79928722,-0.18133595,-0.95824164,-0.55596050,2.41678554,-0.25158118,-0.28234398,-0.15525459,-0.32955230,0.32955230,1077523,Lower East Side,014679/18_115084,1002600001,014679/18,115084,630 WATER STREET,05E,2019-03-15,MANHATTAN,10002,40.71138100,-73.98286600,3.00000000,2.00000000,202.00000000,Lower East Side,2019,2019-03,56.72749658,0.60000000,0.00073801,61.00000000
1,1.21444407,2.18244007,1.64893560,1.26561786,1.40471388,0.99501973,-0.32998824,0.51460360,-0.25457296,-0.86331212,-0.11645154,0.43297468,-0.09638869,-0.18987319,-0.15637232,-0.19165351,-0.07284294,-0.20015991,-0.14005238,-0.11639498,-0.20184605,1.49664288,0.79256320,1.29351915,0.18875881,-0.11971251,-0.00357884,0.52642187,-0.15569947,-0.07392647,-0.15436581,0.03881243,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.38793712,-0.79251171,-1.29195986,-1.22414657,0.28882161,-0.63052791,-0.22514595,-1.42864940,-0.86031694,-1.56374299,0.40252163,-2.33941623,0.29181198,-0.53719203,1.02689236,-1.34858768,-1.35328275,-1.00784528,-0.61702397,0.38290781,-0.25158118,1.00217679,-0.28518706,-1.55423998,1.55423998,1078059,Lower East Side,015506/17_112423,1003560001,015506/17,112423,711 FDR DR,11E,2018-08-22,MANHATTAN,10009,40.72071700,-73.97489500,3.00000000,2.00000000,20.00000000,Lower East Side,2018,2018-08,56.72749658,0.80000000,0.00042988,67.00000000
2,3.14068707,0.24364720,0.29886017,-1.28156686,1.40471388,1.08052701,-0.86390326,0.51460360,-0.25457296,-0.86331212,0.03187075,-0.11418247,-0.08510751,-0.18987319,-0.11861649,-0.05310141,-0.04591595,-0.14915725,-0.08878199,-0.11639498,-0.10638004,-0.13925455,-0.13603531,-0.15644525,-0.09184072,-0.10232200,-0.06181200,-0.03940139,-0.12869400,-0.10686078,-0.12832772,-0.11953955,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.38793712,-0.79251171,-1.29195986,-1.22414657,0.28882161,-0.63052791,-0.22514595,-1.42864940,-0.86031694,-1.56374299,0.40252163,-2.33941623,0.29181198,-0.53719203,1.02689236,-1.34858768,-1.35328275,-1.00784528,-0.61702397,0.38290781,-0.25158118,1.00217679,-0.28518706,-1.55423998,1.55423998,1083144,Lower East Side,0300290/22_5780,1003820001,0300290/22,5780,3 HAVEN PLAZA,15D,2023-05-26,MANHATTAN,10009,40.72682400,-73.97598600,3.00000000,2.00000000,28.00000000,Lower East Side,2023,2023-05,56.72749658,0.80000000,0.00215633,67.00000000
3,3.14068707,0.24364720,0.29886017,-1.28156686

In [ ]:
columns_to_move = ['Location', 'primary_key','bbl','court_index_number','docket_number','eviction_address','eviction_apartment_number',
                   'executed_date','borough','zipcode', 'latitude','longitude','community_board','council_district','census_tract','bin','nta','year',
                    'month_year','eviction_count_nta','eviction_count_building','eviction_count_unit', 'eviction_count_zipcode']

In [ ]:
for col in columns_to_move:
    last_column = BIN_df_normalized.pop(col)
    BIN_df_normalized.insert(0, col, last_column)
BIN_df_normalized.head()

,eviction_count_zipcode,eviction_count_unit,eviction_count_building,eviction_count_nta,month_year,year,nta,bin,census_tract,council_district,community_board,longitude,latitude,zipcode,borough,executed_date,eviction_apartment_number,eviction_address,docket_number,court_index_number,bbl,primary_key,Location,numfloors,unitsres,bldgarea,bldgclass,building_type,architectural_style,building_age,building_category,is_condo,is_llc,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white
0,61.00000000,0.00073801,0.60000000,56.72749658,2019-03,2019,Lower East Side,1077523,202.00000000,2.00000000,3.00000000,-73.98286600,40.71138100,10002,MANHATTAN,2019-03-15,05E,630 WATER STREET,115084,014679/18,1002600001,014679/18_115084,Lower East Side,-0.06971794,0.81877905,0.36540890,-1.28156686,-0.71188874,0.99501973,0.00370864,0.51460360,-0.25457296,-0.86331212,0.21727361,-0.09594390,-0.09638869,-0.18987319,-0.17210392,-0.19165351,-0.09752602,-0.20015991,-0.14005238,-0.11639498,-0.20184605,0.22882237,0.47887445,4.19344796,-0.04273580,-0.12502628,0.03645647,0.07376326,-0.18270494,-0.14644529,-0.15436581,-0.04282892,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.38793712,0.33656949,0.39990329,-0.13447419,-0.08025525,0.31459399,0.37685617,-0.56794790,1.07678865,-0.68257460,2.20956395,-1.80374947,0.02659191,1.40547024,1.27492190,-0.79928722,-0.18133595,-0.95824164,-0.55596050,2.41678554,-0.25158118,-0.28234398,-0.15525459,-0.32955230,0.32955230
1,67.00000000,0.00042988,0.80000000,56.72749658,2018-08,2018,Lower East Side,1078059,20.00000000,2.00000000,3.00000000,-73.97489500,40.72071700,10009,MANHATTAN,2018-08-22,11E,711 FDR DR,112423,015506/17,1003560001,015506/17_112423,Lower East Side,1.21444407,2.18244007,1.64893560,1.26561786,1.40471388,0.99501973,-0.32998824,0.51460360,-0.25457296,-0.86331212,-0.11645154,0.43297468,-0.09638869,-0.18987319,-0.15637232,-0.19165351,-0.07284294,-0.20015991,-0.14005238,-0.11639498,-0.20184605,1.49664288,0.79256320,1.29351915,0.18875881,-0.11971251,-0.00357884,0.52642187,-0.15569947,-0.07392647,-0.15436581,0.03881243,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.38793712,-0.79251171,-1.29195986,-1.22414657,0.28882161,-0.63052791,-0.22514595,-1.42864940,-0.86031694,-1.56374299,0.40252163,-2.33941623,0.29181198,-0.53719203,1.02689236,-1.34858768,-1.35328275,-1.00784528,-0.61702397,0.38290781,-0.25158118,1.00217679,-0.28518706,-1.55423998,1.55423998
2,67.00000000,0.00215633,0.80000000,56.72749658,2023-05,2023,Lower East Side,1083144,28.00000000,2.00000000,3.00000000,-73.97598600,40.72682400,10009,MANHATTAN,2023-05-26,15D,3 HAVEN PLAZA,5780,0300290/22,1003820001,0300290/22_5780,Lower East Side,3.14068707,0.24364720,0.29886017,-1.28156686,1.40471388,1.08052701,-0.86390326,0.51460360,-0.25457296,-0.86331212,0.03187075,-0.11418247,-0.08510751,-0.18987319,-0.11861649,-0.05310141,-0.04591595,-0.14915725,-0.08878199,-0.11639498,-0.10638004,-0.13925455,-0.13603531,-0.15644525,-0.09184072,-0.10232200,-0.06181200,-0.03940139,-0.12869400,-0.10686078,-0.12832772,-0.11953955,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.38793712,-0.79251171,-1.29195986,-1.22414657,0.28882161,-0.63052791,-0.22514595,-1.42864940,-0.86031694,-1.56374299,0.40252163,-2.33941623,0.29181198,-0.53719203,1.02689236,-1.34858768,-1.35328275,-1.00784528,-0.61702397,0.38290781,-0.25158118,1.00217679,-0.28518706,-1.55423998,1.55423998
3,67.00000000,0.00215633,0.80000000,56.7274965

In [ ]:
last_column = BIN_df_normalized.pop('bin')
BIN_df_normalized.insert(0, 'bin', last_column)
BIN_df_normalized.head()

,bin,eviction_count_zipcode,eviction_count_unit,eviction_count_building,eviction_count_nta,month_year,year,nta,census_tract,council_district,community_board,longitude,latitude,zipcode,borough,executed_date,eviction_apartment_number,eviction_address,docket_number,court_index_number,bbl,primary_key,Location,numfloors,unitsres,bldgarea,bldgclass,building_type,architectural_style,building_age,building_category,is_condo,is_llc,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white
0,1077523,61.00000000,0.00073801,0.60000000,56.72749658,2019-03,2019,Lower East Side,202.00000000,2.00000000,3.00000000,-73.98286600,40.71138100,10002,MANHATTAN,2019-03-15,05E,630 WATER STREET,115084,014679/18,1002600001,014679/18_115084,Lower East Side,-0.06971794,0.81877905,0.36540890,-1.28156686,-0.71188874,0.99501973,0.00370864,0.51460360,-0.25457296,-0.86331212,0.21727361,-0.09594390,-0.09638869,-0.18987319,-0.17210392,-0.19165351,-0.09752602,-0.20015991,-0.14005238,-0.11639498,-0.20184605,0.22882237,0.47887445,4.19344796,-0.04273580,-0.12502628,0.03645647,0.07376326,-0.18270494,-0.14644529,-0.15436581,-0.04282892,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.38793712,0.33656949,0.39990329,-0.13447419,-0.08025525,0.31459399,0.37685617,-0.56794790,1.07678865,-0.68257460,2.20956395,-1.80374947,0.02659191,1.40547024,1.27492190,-0.79928722,-0.18133595,-0.95824164,-0.55596050,2.41678554,-0.25158118,-0.28234398,-0.15525459,-0.32955230,0.32955230
1,1078059,67.00000000,0.00042988,0.80000000,56.72749658,2018-08,2018,Lower East Side,20.00000000,2.00000000,3.00000000,-73.97489500,40.72071700,10009,MANHATTAN,2018-08-22,11E,711 FDR DR,112423,015506/17,1003560001,015506/17_112423,Lower East Side,1.21444407,2.18244007,1.64893560,1.26561786,1.40471388,0.99501973,-0.32998824,0.51460360,-0.25457296,-0.86331212,-0.11645154,0.43297468,-0.09638869,-0.18987319,-0.15637232,-0.19165351,-0.07284294,-0.20015991,-0.14005238,-0.11639498,-0.20184605,1.49664288,0.79256320,1.29351915,0.18875881,-0.11971251,-0.00357884,0.52642187,-0.15569947,-0.07392647,-0.15436581,0.03881243,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.38793712,-0.79251171,-1.29195986,-1.22414657,0.28882161,-0.63052791,-0.22514595,-1.42864940,-0.86031694,-1.56374299,0.40252163,-2.33941623,0.29181198,-0.53719203,1.02689236,-1.34858768,-1.35328275,-1.00784528,-0.61702397,0.38290781,-0.25158118,1.00217679,-0.28518706,-1.55423998,1.55423998
2,1083144,67.00000000,0.00215633,0.80000000,56.72749658,2023-05,2023,Lower East Side,28.00000000,2.00000000,3.00000000,-73.97598600,40.72682400,10009,MANHATTAN,2023-05-26,15D,3 HAVEN PLAZA,5780,0300290/22,1003820001,0300290/22_5780,Lower East Side,3.14068707,0.24364720,0.29886017,-1.28156686,1.40471388,1.08052701,-0.86390326,0.51460360,-0.25457296,-0.86331212,0.03187075,-0.11418247,-0.08510751,-0.18987319,-0.11861649,-0.05310141,-0.04591595,-0.14915725,-0.08878199,-0.11639498,-0.10638004,-0.13925455,-0.13603531,-0.15644525,-0.09184072,-0.10232200,-0.06181200,-0.03940139,-0.12869400,-0.10686078,-0.12832772,-0.11953955,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.38793712,-0.79251171,-1.29195986,-1.22414657,0.28882161,-0.63052791,-0.22514595,-1.42864940,-0.86031694,-1.56374299,0.40252163,-2.33941623,0.29181198,-0.53719203,1.02689236,-1.34858768,-1.35328275,-1.00784528,-0.61702397,0.38290781,-0.25158118,1.00217679,-0.28518706,-1.55423998,1.55423998
3,1083141,67.00000000,0.00215633,0.80000000,56

In [ ]:
# last_columns = BIN_df_normalized.pop('Location', 'primary_key','bbl','court_index_number','docket_number','eviction_address','eviction_apartment_number','executed_date','borough','BIN','latitude','longitude','community_board','council_district','census_tract','bin','BIN','year',
#                       'month_year','eviction_count_BIN','eviction_count_building','eviction_count_unit', 'eviction_count_BIN')
# BIN_df_normalized.insert(1, last_columns)
# BIN_df_normalized.head()

In [ ]:
BIN_df_normalized.to_csv('/content/drive/My Drive/X999/BIN_df_normalized.csv', index=False)

In [ ]:
final_model = sm.OLS.from_formula(
    "np.log(eviction_count_building + 1) ~ avg_idr + avg_idr_change + ep_afam + income_change + median_income + ep_hisp + ep_hburd + is_llc + ep_pov150 + rpl_themes + unitsres + building_age + total_complaints",
    data=BIN_df_normalized
).fit()
print(final_model.summary())

                                     OLS Regression Results                                    
Dep. Variable:     np.log(eviction_count_building + 1)   R-squared:                       0.345
Model:                                             OLS   Adj. R-squared:                  0.345
Method:                                  Least Squares   F-statistic:                     2687.
Date:                                 Tue, 10 Jun 2025   Prob (F-statistic):               0.00
Time:                                         19:36:09   Log-Likelihood:                -26603.
No. Observations:                                66395   AIC:                         5.323e+04
Df Residuals:                                    66381   BIC:                         5.336e+04
Df Model:                                           13                                         
Covariance Type:                             nonrobust                                         
                       coef    std err  